In [ ]:
!pip install -q transformers

In [ ]:
def initialize():
  import re 
  import warnings
  import numpy as np
  import pandas as pd
  import seaborn as sns
  import tensorflow as tf
  from wordcloud import WordCloud
  from matplotlib import pyplot as plt
  from transformers import TFAutoModel, AutoTokenizer
  from sklearn.feature_extraction.text import CountVectorizer
  from tensorflow.keras.preprocessing.text import Tokenizer
  from tensorflow.keras.preprocessing.sequence import pad_sequences
  train_df = pd.read_csv('train.csv')
  test_df = pd.read_csv('test.csv')

  input_essay = "It is very important for every child to get an education, as education can change their life. It will prepare the child for the harmonious transformation from a student into a responsible citizen. Thus, in order to make a better life for the children, education is mandatory for everyone. Therefore, the government should make education free for everyone. There are also some negative results if the education in colleges and universities is made free. There are some students, after completing their Higher Secondary, who opt for a professional course or vocational course. These students are aware of their aptitudes and have grown up. Some of them are interested in business, others opt for general courses. If education is made free in colleges and universities, then more students will be interested in taking up general courses. This can further result in unemployment, as there will be fewer job opportunities for the huge number of qualified students. The government can tackle this strong competition by adopting different methods of admitting the student. One of the methods is Entrance- Exams, where the students will be allowed to enter on the basis of their merit. The shortage in the job opportunity will lead the educated and highly qualified to settle down in low-level jobs, where they do not get their deserved honour for their capability and hard work. Thus the people will get demoralised and would get diverted from the idea of getting higher education. This would further put the image of free university education down in the eyes of new aspirants."
  i = 0
  test_df.loc[len(test_df.index)] = ["000BAD50D026"+str(i), input_essay ]
  i+=1
  train_df['full_text'] = train_df["full_text"].replace(re.compile(r'[\n\r\t]'), '', regex=True)
  test_df['full_text'] = test_df["full_text"].replace(re.compile(r'[\n\r\t]'), '', regex=True)

  train_df['num_words'] = train_df['full_text'].apply(lambda x: len(x.split()))
  avg_words = round(train_df['num_words'].mean())
  max_words = round(train_df['num_words'].max())
  print('Average length: {}'.format(avg_words))
  print('Max length: {}'.format(max_words))

  warnings.filterwarnings("ignore", category=FutureWarning)
  sns.set_theme()
  %matplotlib inline 

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
test_data = train_df[3850:3911]

In [ ]:
test_data

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
3850,FDB362959DDF,Do you think that student is much better gradu...,3.0,3.0,3.0,3.0,2.5,2.5
3851,FDC94E0FE9C8,Have you ever did something because is on tren...,3.5,3.0,4.0,3.5,3.0,3.5
3852,FDCBCA9B0636,"Dear School Principal,\r\n\r\nI believe that s...",2.5,3.5,3.5,3.0,4.0,3.5
3853,FDE0B653AD74,"When you play activities or an instrument, don...",4.0,3.5,3.5,4.0,4.0,3.5
3854,FDF7C4F4BC4D,The first place i would want to go is the Baha...,2.5,3.0,3.0,3.5,3.0,3.0
...,...,...,...,...,...,...,...,...
3906,FFD29828A873,I believe using cellphones in class for educat...,2.5,3.0,3.0,3.5,2.5,2.5
3907,FFD9A83B0849,"Working alone, students do not have to argue w...",4.0,4.0,4.0,4.0,3.5,3.0
3908,FFDC4011AC9C,"""A problem is a chance for you to do your best...",2.5,3.0,3.0,3.0,3.5,3.0
3909,FFE16D704B16,Many people disagree with Albert Schweitzer's ...,4.0,4.5,4.5,4.0,4.5,4.5


In [ ]:
train_df = train_df[0:2000]

In [ ]:
train_df.shape

(2000, 8)

In [ ]:
train_df.describe().style.background_gradient(cmap='Blues').set_properties(**{'font-family':'Segoe UI'})
train_df['full_text'][5]

"Dear Principal,\r\n\r\nOur school should have a community center. The reasons why, are so students can learn what our community needs, how to make our community better place, and why is community important for students to know. Its a great to have a community center to know how we can make things better.\r\n\r\nStudents think community center takes their time away. but they have to learn what our community needs. students will participate in a group of students making a list what our community needs, therefore students will learn what our community needs! students will present their list of things our community needs! due to that students will be giving extra credit for the ones who have low grades!\r\n\r\nSome students don't participate because their friends say its waste of time. it would not be waste of time when you get to know how our community can be a better place for us. students should know that the program is about our own lives, because if our community is bad well our live

In [ ]:
train_df['full_text'] = train_df["full_text"].replace(re.compile(r'[\n\r\t]'), '', regex=True)
test_df['full_text'] = test_df["full_text"].replace(re.compile(r'[\n\r\t]'), '', regex=True)

In [ ]:
train_df['num_words'] = train_df['full_text'].apply(lambda x: len(x.split()))
avg_words = round(train_df['num_words'].mean())
max_words = round(train_df['num_words'].max())
print('Average length: {}'.format(avg_words))
print('Max length: {}'.format(max_words))

Average length: 430
Max length: 1260


In [ ]:
cv = CountVectorizer(stop_words = 'english')
words = cv.fit_transform(train_df['full_text'])
sum_words = words.sum(axis=0)

words_freq = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)

wordcloud = WordCloud(width = 2000, height = 2000).generate_from_frequencies(dict(words_freq))


In [ ]:
def tokenize():
  tokenizer = Tokenizer(oov_token="<OOV>")
  tokenizer.fit_on_texts(train_df['full_text'])
  word_index = tokenizer.word_index

  train_seq = tokenizer.texts_to_sequences(train_df['full_text'])
  pad_train = pad_sequences(train_seq, maxlen=max_words, truncating='post')

  test_seq = tokenizer.texts_to_sequences(test_df['full_text'])
  pad_test = pad_sequences(test_seq, maxlen=max_words, truncating='post')

In [ ]:
tokenizew

In [ ]:
word_idx_count = len(word_index)
print(word_idx_count)

15798


In [ ]:
X = pad_train
y = [train_df['cohesion'].values, train_df['syntax'].values, train_df['vocabulary'].values,
      train_df['phraseology'].values, train_df['grammar'].values, train_df['conventions'].values]

In [ ]:

def get_lstm_model():

    inputs = tf.keras.layers.Input(shape=(max_words))
    embeddings = tf.keras.layers.Embedding(word_idx_count + 1, 64, input_length=max_words)(inputs)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(embeddings)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True))(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(x)
    
    output1 = tf.keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05), 
                                    bias_initializer='zeros',
                                    kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4), 
                                    bias_regularizer=tf.keras.regularizers.L2(1e-4), 
                                    activity_regularizer=tf.keras.regularizers.L2(1e-5))(x)
    output1 = tf.keras.layers.Dense(32, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05), 
                                    bias_initializer='zeros',
                                    kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4), 
                                    bias_regularizer=tf.keras.regularizers.L2(1e-4), 
                                    activity_regularizer=tf.keras.regularizers.L2(1e-5))(output1)
    output1 = tf.keras.layers.Dense(1, activation='relu', name='cohesion')(output1)
    
    
    output2 = tf.keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05), 
                                    bias_initializer='zeros',
                                    kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4), 
                                    bias_regularizer=tf.keras.regularizers.L2(1e-4), 
                                    activity_regularizer=tf.keras.regularizers.L2(1e-5))(x)
    output2 = tf.keras.layers.Dense(32, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05), 
                                    bias_initializer='zeros',
                                    kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4), 
                                    bias_regularizer=tf.keras.regularizers.L2(1e-4), 
                                    activity_regularizer=tf.keras.regularizers.L2(1e-5))(output2)
    output2 = tf.keras.layers.Dense(1, activation='relu', name='syntax')(output2)
    
    output3 = tf.keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05), 
                                    bias_initializer='zeros',
                                    kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4), 
                                    bias_regularizer=tf.keras.regularizers.L2(1e-4), 
                                    activity_regularizer=tf.keras.regularizers.L2(1e-5))(x)
    output3 = tf.keras.layers.Dense(32, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05), 
                                    bias_initializer='zeros',
                                    kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4), 
                                    bias_regularizer=tf.keras.regularizers.L2(1e-4), 
                                    activity_regularizer=tf.keras.regularizers.L2(1e-5))(output3)
    output3 = tf.keras.layers.Dense(1, activation='relu', name='vocabulary')(output3)
    
    output4 = tf.keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05), 
                                    bias_initializer='zeros',
                                    kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4), 
                                    bias_regularizer=tf.keras.regularizers.L2(1e-4), 
                                    activity_regularizer=tf.keras.regularizers.L2(1e-5))(x)
    output4 = tf.keras.layers.Dense(32, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05), 
                                    bias_initializer='zeros',
                                    kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4), 
                                    bias_regularizer=tf.keras.regularizers.L2(1e-4), 
                                    activity_regularizer=tf.keras.regularizers.L2(1e-5))(output4)
    output4 = tf.keras.layers.Dense(1, activation='relu', name='phraseology')(output4)
    
    output5 = tf.keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05), 
                                    bias_initializer='zeros',
                                    kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4), 
                                    bias_regularizer=tf.keras.regularizers.L2(1e-4), 
                                    activity_regularizer=tf.keras.regularizers.L2(1e-5))(x)
    output5 = tf.keras.layers.Dense(32, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05), 
                                    bias_initializer='zeros',
                                    kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4), 
                                    bias_regularizer=tf.keras.regularizers.L2(1e-4), 
                                    activity_regularizer=tf.keras.regularizers.L2(1e-5))(output5)
    output5 = tf.keras.layers.Dense(1, activation='relu', name='grammar')(output5)  
    
    output6 = tf.keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05), 
                                    bias_initializer='zeros',
                                    kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4), 
                                    bias_regularizer=tf.keras.regularizers.L2(1e-4), 
                                    activity_regularizer=tf.keras.regularizers.L2(1e-5))(x)
    output6 = tf.keras.layers.Dense(32, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05), 
                                    bias_initializer='zeros',
                                    kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4), 
                                    bias_regularizer=tf.keras.regularizers.L2(1e-4), 
                                    activity_regularizer=tf.keras.regularizers.L2(1e-5))(output6)
    output6 = tf.keras.layers.Dense(1, activation='relu', name='conventions')(output6)
    
    model = tf.keras.models.Model(inputs=inputs, outputs=[output1, output2, output3, output4, output5, output6])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss = tf.keras.losses.mean_squared_error
    )
    
    return model

In [ ]:
model = get_lstm_model()

In [ ]:
history = model.fit(X, y, epochs=30, validation_split=0.2, shuffle=True, verbose=1)

In [ ]:
!mkdir -p saved_model2
model.save('saved_model2/my_model')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files
files.download('/content/saved_model2/my_model/variables/variables.data-00000-of-00001')

In [ ]:
!zip -r /content/saved_model2.zip /content/saved_model2/

In [ ]:
files.download('/content/saved_model2.zip')

In [ ]:
initialize()
tokenize()

Average length: 426
Max length: 1250


In [ ]:

def load_model():
  new_model = tf.keras.models.load_model('drive/MyDrive/LSTM_Model/my_model')
  return new_model
#Check its architecture
#new_model.summary()

In [ ]:
saved_model = load_model()

In [ ]:
saved_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1250)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1250, 64)     1011136     ['input_1[0][0]']                
                                                                                                  
 bidirectional (Bidirectional)  (None, 1250, 128)    66048       ['embedding[0][0]']              
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 1250, 64)    41216       ['bidirectional[0][0]']          
 )                                                                                            

In [ ]:
test_df


In [ ]:
input_essay = "It is very important for every child to get an education, as education can change their life. It will prepare the child for the harmonious transformation from a student into a responsible citizen. Thus, in order to make a better life for the children, education is mandatory for everyone. Therefore, the government should make education free for everyone. There are also some negative results if the education in colleges and universities is made free. There are some students, after completing their Higher Secondary, who opt for a professional course or vocational course. These students are aware of their aptitudes and have grown up. Some of them are interested in business, others opt for general courses. If education is made free in colleges and universities, then more students will be interested in taking up general courses. This can further result in unemployment, as there will be fewer job opportunities for the huge number of qualified students. The government can tackle this strong competition by adopting different methods of admitting the student. One of the methods is Entrance- Exams, where the students will be allowed to enter on the basis of their merit. The shortage in the job opportunity will lead the educated and highly qualified to settle down in low-level jobs, where they do not get their deserved honour for their capability and hard work. Thus the people will get demoralised and would get diverted from the idea of getting higher education. This would further put the image of free university education down in the eyes of new aspirants."

In [ ]:
i = 0
test_df.loc[len(test_df.index)] = ["000BAD50D026"+str(i), input_essay ]
i+=1

In [ ]:
test_df

In [ ]:
print(pad_test.shape)

In [ ]:
maxlen = 1250
sequences = tokenizer.texts_to_sequences([""])
data = pad_sequences(sequences, maxlen=maxlen)

In [ ]:
data

In [ ]:
X_test = data
results = new_model.predict(X_test)

In [ ]:
results

In [ ]:
abc = pd.DataFrame()
abc['text_id'] = test_df['text_id']
base = 0.5
for i in range(6):
    abc[i] = pd.Series([base * round(float(x)/base) for x in results[i]])
abc.columns = train_df.drop(columns=['full_text', 'num_words']).columns
abc = abc.clip(upper=pd.Series({'cohesion':4.0, 'syntax':4.0, 'vocabulary':4.0, 'phraseology':4.0, 'grammar':4.0, 'conventions':4.0}), axis=1)
abc['text_id'] = test_df['text_id']

abc.head()

In [ ]:
test_seq = tokenizer.texts_to_sequences(test_df.iloc[-1]['full_text'])
pad_test = pad_sequences(test_seq, maxlen=max_words, truncating='post')

In [ ]:
test_df['full_text']

In [ ]:
print(X_test)

In [ ]:
import math

In [ ]:
X_test = pad_test
results = saved_model.predict(X_test)
lstm_prediction = pd.DataFrame()
lstm_prediction['text_id'] = test_df['text_id']
base = 0.5
for i in range(6):
    lstm_prediction[i] = pd.Series([base * round(float(x)/base) for x in results[i]])
lstm_prediction.columns = train_df.drop(columns=['full_text', 'num_words']).columns
lstm_prediction = lstm_prediction.clip(upper=pd.Series({'cohesion':4.0, 'syntax':4.0, 'vocabulary':4.0, 'phraseology':4.0, 'grammar':4.0, 'conventions':4.0}), axis=1)
lstm_prediction['text_id'] = test_df['text_id']

lstm_prediction.head()

50/50 [==============================] - 12s 155ms/step


,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,4.0,4.0,4.0,4.0,4.0,4.0
1,000BAD50D026,4.0,4.0,4.0,4.0,4.0,4.0
2,00367BB2546B,4.0,4.0,4.0,4.0,4.0,4.0
3,000BAD50D0260,4.0,4.0,4.0,4.0,4.0,4.0


In [ ]:
average = (lstm_prediction.iloc[-1]['cohesion'] + lstm_prediction.iloc[-1]['syntax'] + lstm_prediction.iloc[-1]['vocabulary'] + lstm_prediction.iloc[-1]['phraseology'] + lstm_prediction.iloc[-1]['grammar'] + lstm_prediction.iloc[-1]['conventions'])/6
print(average)

4.0


In [ ]:
lstm_submission.to_csv('lstmsubmission.csv', index=False)